## Imports / Globals

In [1]:
import pickle
import pandas as pd
from os import listdir
from os.path import isfile, join
from tqdm.notebook import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import random
import itertools

np.random.seed(42)
random.seed(42)

In [2]:
# path = '/Users/thomas/Downloads/nturgb+d_skeletons'
path = 'D:\\Datasets\\Motion Privacy\\NTU RGB+D 120\\Skeleton Data'
X_path = 'data/X.pkl'

## Data organization

In [3]:
def load_files():
    # Read the files
    files = [f for f in listdir(path) if isfile(join(path, f))]

    # Get stats for each file based on name
    files_ = []
    for file in files:
        data = {'file': file,
                's': file[0:4],
                'c': file[4:8],
                'p': file[8:12],
                'r': file[12:16],
                'a': file[16:20]
                }
        files_.append(data)

    return files_
files_ = None

In [4]:
# Attempt to load X and Y from pickle before generating them
X = {}
try:
    print('Attempting to load X from pickle')
    with open(X_path, 'rb') as f:
        X = pickle.load(f)
    print('X loaded from pickle')
except:
    print('Could not load X and Y, generating them now')
    
    # Read the files
    files = [f for f in listdir(path) if isfile(join(path, f))]

    # Get stats for each file based on name
    files_ = []
    for file in files:
        data = {'file': file,
                's': file[0:4],
                'c': file[4:8],
                'p': file[8:12],
                'r': file[12:16],
                'a': file[16:20]
                }
        files_.append(data)

    # Generate X and Y
    for file_ in tqdm(files_, desc='Files Parsed', position=0):
        try:
            file = join(path, file_['file'])
            data = open(file, 'r')
            lines = data.readlines()
            frames_count = int(lines.pop(0).replace('\n', ''))
            file_['frames'] = frames_count
        except UnicodeDecodeError: # .DS_Store file
            print('UnicodeDecodeError: ', file)
            continue

        # Add filename as key to X
        X[file_['file']] = []

        # Skip file if 2 actors
        if lines[0].replace('\n', '') != '1': continue

        for f in tqdm(range(frames_count), desc='Frames Parsed', position=1, leave=False):
            try:
                # Get actor count
                actors = int(lines.pop(0).replace('\n', ''))
            
                # Get actor info
                t = lines.pop(0)

                # Get joint count
                joint_count = int(lines.pop(0).replace('\n', ''))

                # Get joint info
                d = []
                for j in range(joint_count):
                    joint = lines.pop(0).replace('\n', '').split(' ')
                    d.extend(joint[0:3])

                # Skip if not 25 joints
                if len(d) != 75: continue

                # Convert to numpy array
                d = np.array(d)

                # Append to X and Y
                X[file_['file']].append(d)
            except:
                break
        
        # Convert to numpy array
        X[file_['file']] = np.array(X[file_['file']], dtype=np.float16)

        # Pad X size to 300 frames (300 is max frames in dataset)
        X[file_['file']] = np.pad(X[file_['file']], ((0, 300-X[file_['file']].shape[0]), (0, 0)), 'constant')


    print('X Generated, saving to pickle...')

    # Save the data
    with open(X_path, 'wb') as f:
        pickle.dump(X, f)

    print('X Saved to pickle')


Attempting to load X from pickle
X loaded from pickle


## Data Generators

In [5]:
same_samples_per_actor = 10
diff_samples_per_actor = 10
train_samples = 0
test_samples = 0

per_actor = True

def data_generator_per_actor(X, same_samples_per_actor=1000, diff_samples_per_actor=1000, train=True, val_split=0.25):
    actor_data = {}
    for file in X:
        actor = int(file[9:12])
        action = int(file[17:20])

        split_threshold = int((1 - val_split) * 60) if train else int(val_split * 60)

        is_train_or_val = action <= split_threshold
        if train != is_train_or_val:
            continue

        if actor not in actor_data:
            actor_data[actor] = []
        if len(X[file]) == 0:
            continue
        actor_data[actor].append(X[file])

    actor_keys = list(actor_data.keys())

    while True:
        for actor in actor_keys:
            samples = []

            for _ in range(same_samples_per_actor):
                same_video1 = random.choice(actor_data[actor])
                same_video2 = random.choice(actor_data[actor])
                samples.append((np.array([same_video1, same_video2]).astype(np.float32), 1))

            while True:
                diff_actor = random.choice(actor_keys)
                if diff_actor != actor:
                    break

            for _ in range(diff_samples_per_actor):
                same_video1 = random.choice(actor_data[actor])
                diff_video = random.choice(actor_data[diff_actor])
                samples.append((np.array([same_video1, diff_video]).astype(np.float32), 0))

            random.shuffle(samples)

            for sample in samples:
                yield sample

def data_generator(X, same_samples=10000, diff_samples=10000, train=True, val_split=0.25):
    actor_data = {}
    for file in X:
        actor = int(file[9:12])
        action = int(file[17:20])

        split_threshold = int((1 - val_split) * 60) if train else int(val_split * 60)

        is_train_or_val = action <= split_threshold
        if train != is_train_or_val:
            continue

        if actor not in actor_data:
            actor_data[actor] = []
        if len(X[file]) == 0:
            continue
        actor_data[actor].append(X[file])

    samples = []
    
    for _ in range(same_samples):
        actor = random.choice(list(actor_data.keys()))
        video1 = random.choice(actor_data[actor])
        video2 = random.choice(actor_data[actor])
        samples.append((np.array([video1, video2]).astype(np.float32), 1))

    for _ in range(diff_samples):
        actor1 = random.choice(list(actor_data.keys()))
        actor2 = random.choice(list(actor_data.keys()))
        while actor1 == actor2:
            actor2 = random.choice(list(actor_data.keys()))
        video1 = random.choice(actor_data[actor1])
        video2 = random.choice(actor_data[actor2])
        samples.append((np.array([video1, video2]).astype(np.float32), 0))
    
    random.shuffle(samples)

    while True:
        for sample in samples:
            yield sample

if per_actor:
    train_gen = data_generator_per_actor(X, same_samples_per_actor, diff_samples_per_actor, train=True, val_split=0.25)
    val_gen = data_generator_per_actor(X, same_samples_per_actor, diff_samples_per_actor, train=False, val_split=0.5)
    test_gen = data_generator_per_actor(X, same_samples_per_actor, diff_samples_per_actor, train=False, val_split=0)
else:
    train_gen = data_generator(X, same_samples_per_actor, diff_samples_per_actor, train=True, val_split=0.25)
    val_gen = data_generator(X, same_samples_per_actor, diff_samples_per_actor, train=False, val_split=0.5)
    test_gen = data_generator(X, same_samples_per_actor, diff_samples_per_actor, train=False, val_split=0)

## SGN

All code in this section is adapted from Microsoft's SGN. [Github](https://github.com/microsoft/SGN)

In [6]:
# Hyperparameters/Tuning Parameters
dataset='NTU'
batch_size=64
max_epochs=20
lr=.5
weight_decay=0.1
do_train=1
seg=20
# load_model='best_model'
load_model=False

In [7]:
import time
import shutil
import os
import os.path as osp
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import Dataset, DataLoader
from model import SGN
from data import AverageMeter#, NTUDataLoaders
from util import make_dir
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import CrossEntropyLoss
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR

In [16]:
# Tweaks for Linkage Attack
class SGN_Linkage_Attack(nn.Module):
    def __init__(self, model_a, model_b, output_size):
        super(SGN_Linkage_Attack, self).__init__()
        self.model_a = model_a
        self.model_b = model_b
        self.fc = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.25),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(128, output_size),
            nn.Sigmoid()
        )
    
    def forward(self, x_a, x_b):
        a_out = self.model_a(x_a).cuda()
        b_out = self.model_b(x_b).cuda()
        out = torch.cat((a_out, b_out), dim=1)
        out = self.fc(out)
        return out
    
    def print_weights(self):
        print('FC Weights: ', self.fc[0].weight)
        print('FC Bias: ', self.fc[0].bias)
        print('Model A state_dict:', self.model_a.state_dict())
        print('Model B state_dict:', self.model_b.state_dict())

class test_model(nn.Module):
    def __init__(self):
        super(test_model, self).__init__()
        self.lstm_a = nn.LSTM(75, 256, num_layers=2, batch_first=True, dropout=0.5)
        self.lstm_b = nn.LSTM(75, 256, num_layers=2, batch_first=True, dropout=0.5)
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x_a, x_b):
        x_a, _ = self.lstm_a(x_a)
        x_b, _ = self.lstm_b(x_b)
        x_a = x_a[:, -1, :]
        x_b = x_b[:, -1, :]
        x = torch.cat([x_a, x_b], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

class SGN_Linkage_Dataset(Dataset):
    def __init__(self, data_gen, dataset_len, seg=20):
        self.data_gen = data_gen
        self.len = dataset_len
        self.seg = seg

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        x, y = next(self.data_gen) 
        x_a, x_b = self.preprocess(x)
        return x_a, x_b, y

    def preprocess(self, x):
        x_a = x[0]
        x_b = x[1]

        x_a, _ = self.tolist_fix([x_a], [0], train=1)
        x_a = torch.tensor(x_a[0]).cuda()
        x_b, _ = self.tolist_fix([x_b], [0], train=1)
        x_b = torch.tensor(x_b[0]).cuda()

        # epsilon = 1e-8
        # x_a = (x_a - x_a.mean(dim=1, keepdim=True)) / (x_a.std(dim=1, keepdim=True) + epsilon)
        # x_b = (x_b - x_b.mean(dim=1, keepdim=True)) / (x_b.std(dim=1, keepdim=True) + epsilon)

        return x_a, x_b

    def tolist_fix(self, joints, y, train=1):
        seqs = []

        for idx, seq in enumerate(joints):
            zero_row = []
            for i in range(len(seq)):
                if np.array_equal(seq[i, :], np.zeros(75)):
                    zero_row.append(i)

            seq = np.delete(seq, zero_row, axis=0)
            seqs = self.sub_seq(seqs, seq, train=train)

        return seqs, y

    def sub_seq(self, seqs, seq, train=1):
        group = self.seg

        if seq.shape[0] < self.seg:
            pad = np.zeros(
                (self.seg - seq.shape[0], seq.shape[1])).astype(np.float32)
            seq = np.concatenate([seq, pad], axis=0)

        ave_duration = seq.shape[0] // group

        if train == 1:
            offsets = np.multiply(
                list(range(group)), ave_duration) + np.random.randint(ave_duration, size=group)
            seq = seq[offsets]
            seqs.append(seq)

        return seqs

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            target = target.long()  # Convert target tensor to long
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

class FocalLoss(nn.Module):
    def __init__(self, num_classes, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.num_classes = num_classes
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        # Move targets to the same device as the indexed tensor (if not already)
        targets = targets.to(inputs.device)

        # Convert targets to one-hot encoding
        one_hot_targets = torch.eye(self.num_classes, device=inputs.device)[targets]

        # Compute the softmax of the inputs
        probabilities = F.softmax(inputs, dim=1)

        # Compute the focal loss
        pt = torch.where(one_hot_targets == 1, probabilities, 1 - probabilities)
        ce_loss = F.cross_entropy(inputs, targets, reduction='none').unsqueeze(1)
        focal_loss = self.alpha * (1 - pt)**self.gamma * ce_loss

        return focal_loss.mean()

class CategoricalHingeLoss(nn.Module):
    def __init__(self, num_classes):
        super(CategoricalHingeLoss, self).__init__()
        self.num_classes = num_classes

    def forward(self, inputs, targets):
        # Convert targets to one-hot encoding
        targets_one_hot = torch.zeros(targets.shape[0], self.num_classes).cuda()
        targets_one_hot.scatter_(1, targets.unsqueeze(1), 1)
        pos_scores = (inputs * targets_one_hot).sum(dim=1)
        neg_scores = (inputs * (1 - targets_one_hot)).sum(dim=1)
        hinge_loss = torch.clamp(1 + neg_scores - pos_scores, min=0)
        return hinge_loss.mean()

def evaluate(model, criterion, validation_loader):
    model.eval()
    total_loss = 0
    total_samples = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for x_a, x_b, targets in tqdm(validation_loader, leave=False, desc='Validation', position=1):
            x_a, x_b, targets = x_a.cuda(), x_b.cuda(), targets.float().cuda()

            outputs = model(x_a, x_b)
            loss = criterion(outputs.squeeze(), targets)

            total_loss += loss.item() * targets.size(0)
            total_samples += targets.size(0)

            predictions = (torch.sigmoid(outputs) > 0.5).long().squeeze()  # Convert outputs to binary format
            all_targets.extend(targets.tolist())
            all_predictions.extend(predictions.tolist())

    # Calculate validation loss and accuracy
    val_loss = total_loss / total_samples
    val_accuracy = accuracy_score(all_targets, all_predictions)

    return val_loss, val_accuracy


def evaluate_metrics(model, criterion, data_loader):
    model.eval()
    total_loss = 0
    total_samples = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for x_a, x_b, targets in tqdm(data_loader, leave=False, desc='Testing', position=1):
            x_a, x_b, targets = x_a.cuda(), x_b.cuda(), targets.cuda()

            outputs = model(x_a, x_b)
            loss = criterion(outputs, targets)

            total_loss += loss.item() * targets.size(0)
            total_samples += targets.size(0)

            _, predictions = torch.max(outputs, 1)  # Convert one-hot encoded predictions to binary format
            all_targets.extend(targets.tolist())
            all_predictions.extend(predictions.tolist())

    # Calculate metrics
    loss = total_loss / total_samples
    accuracy = accuracy_score(all_targets, all_predictions)
    cm = confusion_matrix(all_targets, all_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_predictions, average='binary')

    return accuracy, loss, cm, precision, recall, f1


In [17]:
def main(do_train = True):
    # Initiate the two SGN models
    num_classes = 1 # Is the same or not the same
    SGN_Encoder1 = SGN(num_classes, dataset, seg, batch_size, do_train).cuda()
    SGN_Encoder2 = SGN(num_classes, dataset, seg, batch_size, do_train).cuda()
    
    # Combine the two SGN Models
    # model = SGN_Linkage_Attack(SGN_Encoder1, SGN_Encoder2, num_classes).cuda()
    model = test_model().cuda()

    # Load the data and create dataloaders
    num_actors = len(set(int(file[9:12]) for file in X))
    samples_per_actor = same_samples_per_actor + diff_samples_per_actor
    if per_actor:
        train_len = int(samples_per_actor * num_actors * 0.5)
        val_len = int(samples_per_actor * num_actors * 0.25)
        test_len = int(samples_per_actor * num_actors * 0.25)
    else:
        train_len = int(samples_per_actor * 0.5)
        val_len = int(samples_per_actor * 0.25)
        test_len = int(samples_per_actor * 0.25)

    train_dataset = SGN_Linkage_Dataset(train_gen, train_len)
    val_dataset = SGN_Linkage_Dataset(val_gen, val_len)
    test_dataset = SGN_Linkage_Dataset(test_gen, test_len)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

    # Set up checkpoint director
    checkpoint_dir = 'models'
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Load the model if specified
    if load_model:
        # Load the model
        model.load_state_dict(torch.load(f'{checkpoint_dir}/{load_model}.pt'))
        print('Model loaded')

    # Initialize variables for tracking loss
    best_loss = float('inf')
    best_epoch = 0

    # Train the model
    # criterion = LabelSmoothingLoss(num_classes, smoothing=0.1).cuda()
    criterion = nn.BCELoss().cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    if do_train:
        for epoch in tqdm(range(max_epochs), desc='Epochs'):
            # Train
            model.train()
            optimizer.zero_grad()
            total_train_loss = 0  # Add this line to store total training loss
            for i, (x_a, x_b, y) in enumerate(tqdm(train_loader, leave=False, desc='Training', position=1)):
                x_a = x_a.cuda()
                x_b = x_b.cuda()
                y = y.float().cuda()

                output = model(x_a, x_b)
                loss = criterion(output.squeeze(), y)

                total_train_loss += loss.item()  # Update total training loss
                loss.backward()

                # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

                optimizer.step()
                optimizer.zero_grad()

            avg_train_loss = total_train_loss / len(train_loader)  # Calculate average training loss

            # Evaluate
            val_loss, val_acc = evaluate(model, criterion, validation_loader)
            print(f'Epoch {epoch+1}, Training Loss: {avg_train_loss}, Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')

            # Save the best model
            if val_loss < best_loss:
                best_loss = val_loss
                best_epoch = epoch
                torch.save(model.state_dict(), os.path.join(checkpoint_dir, 'best_model.pt'))
                print(f'New best validation loss, checkpoint saved')

    # Load the best model
    model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'best_model.pt')))

    # Evaluate the model
    accuracy, loss, cm, precision, recall, f1 = evaluate_metrics(model, criterion, test_loader)
    print(f'Accuracy: {accuracy}')
    print(f'Loss: {loss}')
    print(f'Confusion Matrix: {cm}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1: {f1}')

main(do_train=True)

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Training:   0%|          | 0/16 [00:00<?, ?it/s]

Validation:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1, Training Loss: 0.6939134784042835, Validation Loss: 0.6933486461639404, Validation Accuracy: 0.498046875
New best validation loss, checkpoint saved


Training:   0%|          | 0/16 [00:00<?, ?it/s]

...

Training:   0%|          | 0/16 [00:00<?, ?it/s]

Validation:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 20, Training Loss: 0.6931507736444473, Validation Loss: 0.6931337714195251, Validation Accuracy: 0.49609375


Testing:   0%|          | 0/8 [00:00<?, ?it/s]

ValueError: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.